In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()
none_age = train_data.loc[train_data.Cabin.notna()]
print(none_age)

     PassengerId  Survived  Pclass  \
1              2         1       1   
3              4         1       1   
6              7         0       1   
10            11         1       3   
11            12         1       1   
..           ...       ...     ...   
871          872         1       1   
872          873         0       1   
879          880         1       1   
887          888         1       1   
889          890         1       1   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                              McCarthy, Mr. Timothy J    male  54.0      0   
10                     Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                            Bonnell, Miss. Elizabeth  female  58.0      0   
..                                                 ...     ...   ... 

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
submit = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
submit.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# Load the dataset
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

# Select features and target
features = ['Pclass', 'Sex', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']
X_features = train_data[features]
X_labels = train_data['Survived']

# check which columsn have null values
naColumns = X_features.columns[X_features.isna().any()].tolist()
print(naColumns)

['Age', 'Cabin', 'Embarked']


In [6]:
# Fill missing values
X_features.loc[:,'Age'] = pd.to_numeric(X_features['Age'], errors='coerce')
X_features.loc[X_features.Age.isna(), 'Age'] = X_features['Age'].median()

X_features.loc[X_features.Embarked.isna(), 'Embarked'] = X_features['Embarked'].mode()[0]
X_features.loc[X_features.Cabin.isna(), 'Cabin'] = X_features['Cabin'].mode()[0]
# check which columsn have null values
naColumns = X_features.columns[X_features.isna().any()].tolist()
print(naColumns)

[]


In [7]:
# Preprocessing for numerical and categorical data
preprocessor = make_column_transformer(
    (StandardScaler(), ['Age', 'Fare', 'SibSp', 'Parch']),
    (OneHotEncoder(), ['Pclass', 'Sex', 'Embarked'])
)

In [8]:
# Split the data
features_train, features_valid, labels_train, labels_valid = train_test_split(X_features, X_labels, test_size=0.2, random_state=0)

features_train.head()
type(features_train)

# Create a preprocessing and modeling pipeline
pipeline = make_pipeline(preprocessor)
features_train = pipeline.fit_transform(features_train)
features_valid = pipeline.transform(features_valid)

# type(features_train)
type(features_valid)

numpy.ndarray

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

# Build the model
model = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

2024-04-05 06:35:27.465900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 06:35:27.466105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 06:35:27.602693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(
    features_train, labels_train,
    validation_data=(features_valid, labels_valid),
    batch_size=32,
    epochs=100,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6740 - loss: 0.6314 - val_accuracy: 0.7486 - val_loss: 0.5394
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7207 - loss: 0.5696 - val_accuracy: 0.7933 - val_loss: 0.4882
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7902 - loss: 0.5090 - val_accuracy: 0.7933 - val_loss: 0.4625
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7764 - loss: 0.5256 - val_accuracy: 0.7877 - val_loss: 0.4406
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8069 - loss: 0.4839 - val_accuracy: 0.7933 - val_loss: 0.4237
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8114 - loss: 0.4974 - val_accuracy: 0.7989 - val_loss: 0.4188
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8049 - loss: 0.4891 - val_accuracy: 0.7933 - val_loss: 0.4098
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8072 - loss: 0.4465 - val_accuracy: 0.8045 - 

In [11]:
loss, accuracy = model.evaluate(features_valid, labels_valid)
print("Validation accuracy: {:.2f}%".format(accuracy * 100))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8299 - loss: 0.3698 
Validation accuracy: 82.12%


In [12]:
test_data_features = test_data[features]
#predictions = model.predict(test_test)
print(test_data_features)

     Pclass     Sex                                          Name   Age  \
0         3    male                              Kelly, Mr. James  34.5   
1         3  female              Wilkes, Mrs. James (Ellen Needs)  47.0   
2         2    male                     Myles, Mr. Thomas Francis  62.0   
3         3    male                              Wirz, Mr. Albert  27.0   
4         3  female  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  22.0   
..      ...     ...                                           ...   ...   
413       3    male                            Spector, Mr. Woolf   NaN   
414       1  female                  Oliva y Ocana, Dona. Fermina  39.0   
415       3    male                  Saether, Mr. Simon Sivertsen  38.5   
416       3    male                           Ware, Mr. Frederick   NaN   
417       3    male                      Peter, Master. Michael J   NaN   

     SibSp  Parch      Fare Cabin Embarked  
0        0      0    7.8292   NaN        Q  
1        

In [13]:
# Fill missing values
test_data_features.loc[test_data_features.Age.isna(), 'Age'] = test_data_features['Age'].median()
test_data_features.loc[test_data_features.Fare.isna(), 'Fare'] = test_data_features['Fare'].median()
test_data_features.loc[test_data_features.Embarked.isna(), 'Embarked'] = test_data_features['Embarked'].mode()[0]
test_data_features.loc[test_data_features.Cabin.isna(), 'Cabin'] = test_data_features['Cabin'].mode()[0]
# check which columsn have null values
naColumns = test_data_features.columns[test_data_features.isna().any()].tolist()
print(naColumns)

[]


In [14]:

test_data_features_array = pipeline.transform(test_data_features)
predictions = model.predict(test_data_features_array)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [15]:
# type(predictions)
for index, value in enumerate(predictions):
    print(f"Index {index}: {value}")

Index 0: [0.0926135]
Index 1: [0.2861439]
Index 2: [0.05064818]
Index 3: [0.11944515]
Index 4: [0.5325297]
Index 5: [0.17278355]
Index 6: [0.7897046]
Index 7: [0.24011083]
Index 8: [0.72135574]
Index 9: [0.11227312]
Index 10: [0.11859364]
Index 11: [0.23239425]
Index 12: [0.94048125]
Index 13: [0.0660352]
Index 14: [0.93351287]
Index 15: [0.92793936]
Index 16: [0.11443063]
Index 17: [0.18732734]
Index 18: [0.37814215]
Index 19: [0.50058925]
Index 20: [0.38923767]
Index 21: [0.29933417]
Index 22: [0.92433244]
Index 23: [0.5468588]
Index 24: [0.9163027]
Index 25: [0.06394044]
Index 26: [0.9547673]
Index 27: [0.17560434]
Index 28: [0.26110154]
Index 29: [0.11274834]
Index 30: [0.08603699]
Index 31: [0.1525877]
Index 32: [0.33492213]
Index 33: [0.44758263]
Index 34: [0.45798796]
Index 35: [0.20581615]
Index 36: [0.4604168]
Index 37: [0.53678876]
Index 38: [0.12575606]
Index 39: [0.16897625]
Index 40: [0.11501105]
Index 41: [0.3160962]
Index 42: [0.08523271]
Index 43: [0.8938754]
Index 44: 

In [16]:
predictions = np.round(predictions).astype(int).reshape(-1)
print(predictions)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 1 1 1 1 0 0 1 0 0 0]


In [17]:
# Prepare submission
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': predictions})
submission.to_csv('submission.csv', index=False)

In [18]:
print(submission)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
